In [1]:
from clean_colored_lyrics import *
from synced_lyrics import *
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import regex
import datetime
import pandas as pd

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
song = 'super'
artist = 'seventeen'

raw_color_coded = get_raw_color_coded_html(song)
colored_lyrics = get_colored_lyrics(raw_color_coded)


export_testing_html(str(raw_color_coded['main_lyrics_body']))

for c in colored_lyrics:
    print(c)

url found: https://colorcodedlyrics.com/2023/04/24/seventeen-super-sonogong/
[<span style="color: #4ea8d4">S.Coups</span>, <span style="color: #e32636">Jeonghan</span>, <span style="color: #df73ff">Joshua</span>, <span style="color: #bede0d">Jun</span>, <span style="color: #c04737">Hoshi</span>, <span style="color: #855fc4">Wonwoo</span>, <span style="color: #ff7d07">Woozi</span>, <span style="color: #3d72e7">DK</span>, <span style="color: #04a497">Mingyu</span>, <span style="color: #a1dad7">THE8</span>, <span style="color: #f1ce5f">Seungkwan</span>, <span style="color: #63b76c">Vernon</span>, <span style="color: #d962b6">Dino</span>]
{'member': ['Woozi'], 'lyric': 'SEVENTEEN right here', 'section': 0, 'line': 0}
{'member': ['Wonwoo'], 'lyric': '땅을 보고 계속 올랐지 정상까지', 'section': 1, 'line': 0}
{'member': ['Wonwoo'], 'lyric': '많은 시련은 보란 듯이 I Always Win', 'section': 1, 'line': 1}
{'member': ['Wonwoo'], 'lyric': '강한 마음이 중요하지 ', 'section': 1, 'line': 2}
{'member': ['Wonwoo'], 'lyric': '미래는 도망가

In [ ]:
# song_info = get_song_info(song, artist)
# synced_lyrics = song_info["syncedLyrics"]

In [4]:
for s in synced_lyrics:
    print(s)

{'start': '00:07.43', 'end': '00:13.29', 'lyric': 'Ooh-ooh-ah-ah-ah, ah-ah-ah-ah, yeah', 'delta': '00:05.86'}
{'start': '00:13.29', 'end': '00:14.38', 'lyric': 'SEVENTEEN right here', 'delta': '00:01.09'}
{'start': '00:14.38', 'end': '00:18.07', 'lyric': '땅을 보고 계속 올랐지 정상까지', 'delta': '00:03.69'}
{'start': '00:18.07', 'end': '00:21.35', 'lyric': '많은 시련은 보란 듯이, I always win', 'delta': '00:03.28'}
{'start': '00:21.35', 'end': '00:23.80', 'lyric': '강한 마음이 중요하지 (중요하지)', 'delta': '00:02.45'}
{'start': '00:23.80', 'end': '00:29.23', 'lyric': '미래는 도망가지 않아 내가 놓기 전까지', 'delta': '00:05.43'}
{'start': '00:29.23', 'end': '00:32.84', 'lyric': 'Da-rum-da-rim-da, 구름을 타고 여기저기로 (hey)', 'delta': '00:03.61'}
{'start': '00:32.84', 'end': '00:36.46', 'lyric': 'Da-rum-da-rim-da, 우리들의 긍지를 높이러 (hey)', 'delta': '00:03.62'}
{'start': '00:36.46', 'end': '00:40.29', 'lyric': 'Da-rum-da-rim-da, 또 다른 세계 나아가자, go', 'delta': '00:03.83'}
{'start': '00:40.29', 'end': '00:42.38', 'lyric': '이 rhythm에 맞춰', 'delta': '00:02.

In [5]:
def combine_lines(longer_line, shorter_lines, longer_line_is_official):
    index = 0
    line = ''
    found = False
    index_found = 0
    members = set()
    first_line = []
    is_first_line = True

    # print('combine', longer_line, shorter_lines)
    while longer_line and index < len(shorter_lines):
        shorter_line = shorter_lines[index]
        shorter_norm = normalize_lyrics(shorter_line['lyric'].lower())
            
        if shorter_norm in longer_line or fuzz.partial_ratio(shorter_norm,longer_line) > 0.95:
            index_found += 1

            print("found", shorter_norm)
            if not found:
                found = True
                
            if is_first_line:
                first_line = shorter_line
                is_first_line = False
            
            longer_line = longer_line.replace(shorter_norm, '', 1).strip()

            if not longer_line_is_official:
                line += (shorter_line['lyric'] + ' ')
                for m in shorter_line['member']:
                    members.add(m)
            
            if longer_line == '':
                found = True
                break
        index += 1
          
    if longer_line_is_official:
        line = longer_line
    line.strip()

    return {'index': index_found, 'line': line, 'found': found, 'members': list(members), 'first_line': first_line}

In [20]:
def merge_color_into_synced(synced, shorter_lines):
    longer_line = normalize_lyrics(synced['lyric'].lower().strip())
    original_longer_line = longer_line
    index = 0
    line = ''
    found = False
    index_found = 0
    first_line = []
    is_first_line = True
    current_member = ''
    next_member = ''
    lines_found = []

    # print('combine', longer_line, shorter_lines)
    while longer_line and index < len(shorter_lines):
        shorter_line = shorter_lines[index]
        shorter_norm = normalize_lyrics(shorter_line['lyric'].lower())
        current_member = ''
        for m in shorter_line['member']:
            current_member += (m + ' ')

        print('shorter_norm',shorter_norm)
        
        # if line found
        if fuzz.partial_ratio(shorter_norm,longer_line) > 95:
            if not found:
                found = True

            for m in shorter_line['member']:
                next_member += (m + ' ')

            if next_member != current_member:
                ratio = float(len(line))/ float(len(original_longer_line))
                delta_time = str_to_time(synced['delta'], '.')

                delta = datetime.timedelta(minutes=delta_time.minute, seconds=delta_time.second, microseconds=delta_time.microsecond)
                new_delta = delta.total_seconds() - delta.total_seconds() * 0.5
                new_end_time = datetime.timedelta(seconds=new_delta)
                start_time = str_to_time(synced['end'], '.')

                lines_found.append({
                    'member': current_member,
                    'line': line,
                    'start': synced['start'],
                    'end': time_to_string(str(new_end_time)),
                    'delta': new_delta
                })
                synced['start'] = time_to_string(str(new_end_time))
                synced['delta'] = getDelta(synced,'.')
                current_member = next_member
            else:
                line += (shorter_line['lyric'] + ' ')

            
            if is_first_line:
                first_line = shorter_line
                is_first_line = False
            

            # Reduce longer line
            match = regex.search(f"(?b)({shorter_norm}){{e<=5}}", longer_line)
            if match:
                start, end = match.span()
                longer_line = longer_line[:start] + longer_line[end:]
                print("longer_line new", longer_line)
            
            if longer_line == '':
                break

            index_found += 1
        # If next line is not a match, end
        elif found: 
            lines_found.append({
                'member': current_member,
                'line': line,
                'start': synced['start'],
                'end': synced['end'],
                'delta': synced['delta']
            })
            break

        index += 1
          
    line = longer_line.strip()
    return {'index': index_found, 'lines_found': lines_found, 'first_line': first_line}

In [21]:
merged_data = []

colored_index = 0
synced_index = 0


print(len(colored_lyrics), len(synced_lyrics))

while colored_index < len(colored_lyrics) and synced_index < len(synced_lyrics):
    synced = synced_lyrics[synced_index]
    print(synced)
    synced_norm = normalize_lyrics(synced['lyric'].lower().strip())
    matched_members = ['Unknown']
    matched_section = -1
    matched_line = -1
    colored_lyric = 'Unknown'
    did_find = False

    temp_index = 0

    for look_ahead in range(5):
        if colored_index + look_ahead >= len(colored_lyrics):
            break

        colored_data = colored_lyrics[colored_index+look_ahead]

        colored_lyric = colored_data['lyric']
        colored_norm = normalize_lyrics(colored_lyric.lower().strip())
        print(colored_data)

        # IF EXACTLY THE SAME
        if colored_norm == synced_norm:
            matched_members = colored_data['member']
            matched_section = colored_data['section']
            matched_line = colored_data['line']
            colored_index += (look_ahead + 1)
            synced_index += (temp_index + 1)
            print('match')
            did_find = True
            break
        
        else:
            check_further = 5

            # SYNC IS LONGER AND MERGINING COLORED LINES
            if len(synced_norm) > len(colored_norm):
                print('SYNC LONGER')
                if check_further + colored_index >= len(colored_lyrics):
                    check_further = len(colored_lyrics) - check_further - 1

                new_colored_lines = colored_lyrics[colored_index : colored_index + check_further].copy()
                combined_results = merge_color_into_synced(synced, new_colored_lines)
                print('combined_results', combined_results)
                temp_index += 1
                if combined_results['lines_found']:
                    for l in combined_results['lines_found']:
                        add_line = {
                            'start': synced['start'],
                            'end': synced['end'],
                            'lyric': synced['lyric'],
                            'colored_lyric': colored_lyric,
                            'members': matched_members,
                            'section': matched_section,
                            'line': matched_line,
                        }
                        print("ADD LINE;", add_line)
                        merged_data.append(add_line)
                    # colored_index += combined_results['index']
                    # final_lyric = combined_results['line']
                    # matched_members = combined_results['members']
                    # matched_section = combined_results['first_line']['section']
                    # matched_line = combined_results['first_line']['line']
                    did_find = True
                    break
                else:
                    # colored_index += 1
                    final_lyric = synced['lyric']

                
            else:
                print('COLOR LONGER')
                if check_further + synced_index >= len(synced_lyrics):
                    check_further = len(synced_lyrics) - check_further - 1

                combined_results = combine_lines(colored_norm, synced_lyrics[synced_index:synced_index+check_further], True)

                colored_index += 1
                if combined_results['found']:
                    synced_index += combined_results['index']
                    final_lyric = combined_results['line']
                    did_find = True
                    break
                else:
                    # synced_index += 1
                    final_lyric = colored_data['lyric']

    # print(colored_index, synced_index, {
    #         'start': synced['start'],
    #         'end': synced['end'],
    #         'lyric': synced['lyric'],
    #         'colored_lyric': colored_lyric,
    #         'members': matched_members,
    #         'section': matched_section,
    #         'line': matched_line,
    #     })
    
    if temp_index == 5:
        synced_index += 1
        colored_lyric = 'UNKNOWN'
    else:
        synced_index += temp_index

    
    
    merged_data.append({
        'start': synced['start'],
        'end': synced['end'],
        'lyric': synced['lyric'],
        'colored_lyric': colored_lyric,
        'members': matched_members,
        'section': matched_section,
        'line': matched_line,
    })

    
    print()

77 50
{'start': '00:07.43', 'end': '00:13.29', 'lyric': 'Ooh-ooh-ah-ah-ah, ah-ah-ah-ah, yeah', 'delta': '00:05.86'}
{'member': ['Woozi'], 'lyric': 'SEVENTEEN right here', 'section': 0, 'line': 0}
SYNC LONGER
shorter_norm seventeenrighthere
shorter_norm 땅을보고계속올랐지정상까지
shorter_norm 많은시련은보란듯이ialwayswin
shorter_norm 강한마음이중요하지
shorter_norm 미래는도망가지않아내가놓기전까지
combined_results {'index': 0, 'lines_found': [], 'first_line': []}
{'member': ['Wonwoo'], 'lyric': '땅을 보고 계속 올랐지 정상까지', 'section': 1, 'line': 0}
SYNC LONGER
shorter_norm seventeenrighthere
shorter_norm 땅을보고계속올랐지정상까지
shorter_norm 많은시련은보란듯이ialwayswin
shorter_norm 강한마음이중요하지
shorter_norm 미래는도망가지않아내가놓기전까지
combined_results {'index': 0, 'lines_found': [], 'first_line': []}
{'member': ['Wonwoo'], 'lyric': '많은 시련은 보란 듯이 I Always Win', 'section': 1, 'line': 1}
SYNC LONGER
shorter_norm seventeenrighthere
shorter_norm 땅을보고계속올랐지정상까지
shorter_norm 많은시련은보란듯이ialwayswin
shorter_norm 강한마음이중요하지
shorter_norm 미래는도망가지않아내가놓기전까지
combined_results {'index': 0, 'line

In [16]:
print(len(merged_data))
for m in merged_data:
    print(m)

46
{'start': '00:07.43', 'end': '00:13.29', 'lyric': 'Ooh-ooh-ah-ah-ah, ah-ah-ah-ah, yeah', 'colored_lyric': 'UNKNOWN', 'members': ['Unknown'], 'section': -1, 'line': -1}
{'start': '00:13.29', 'end': '00:14.38', 'lyric': 'SEVENTEEN right here', 'colored_lyric': 'SEVENTEEN right here', 'members': ['Woozi'], 'section': 0, 'line': 0}
{'start': '00:14.38', 'end': '00:18.07', 'lyric': '땅을 보고 계속 올랐지 정상까지', 'colored_lyric': '땅을 보고 계속 올랐지 정상까지', 'members': ['Wonwoo'], 'section': 1, 'line': 0}
{'start': '00:18.07', 'end': '00:21.35', 'lyric': '많은 시련은 보란 듯이, I always win', 'colored_lyric': '많은 시련은 보란 듯이 I Always Win', 'members': ['Wonwoo'], 'section': 1, 'line': 1}
{'start': '00:21.35', 'end': '00:23.80', 'lyric': '강한 마음이 중요하지 (중요하지)', 'colored_lyric': '강한 마음이 중요하지 ', 'members': ['Unknown'], 'section': -1, 'line': -1}
{'start': '00:21.35', 'end': '00:23.80', 'lyric': '강한 마음이 중요하지 (중요하지)', 'colored_lyric': '강한 마음이 중요하지 ', 'members': ['Unknown'], 'section': -1, 'line': -1}
{'start': '00:02:71', 'e

In [ ]:
synced = synced_lyrics[0]

delta_time = str_to_time(synced['delta'], ':')
                
delta = timedelta(minutes=delta_time.minute, seconds=delta_time.second, microseconds=delta_time.microsecond)
old_delta = delta.total_seconds() - delta.total_seconds() * 0.5
new_end_time = datetime.timedelta(seconds=old_delta)
start_time = str_to_time(synced['end'], '.')
print(type(synced['end']))
print( delta.total_seconds() ,old_delta)
pd.to_datetime(old_delta, unit='s')

<class 'str'>
5.86 2.93


Timestamp('1970-01-01 00:00:02.930000')